In [1]:
#!python -m spacy download pt_core_news_lg

In [2]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy

### Specify Stardog connection details

In [3]:
# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

In [4]:
database_name = 'IndigenousSlavery'
conn = stardog.Connection(database_name, **connection_details)

In [110]:
query = """
SELECT ?thesis ?abstract (lang(?abstract) AS ?lang) WHERE {
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <http://purl.org/ontology/bibo/abstract> ?abstract.
  FILTER (langMatches(lang(?abstract),"pt"))
}
LIMIT 20
"""
#  FILTER (langMatches(lang(?abstract),"en"))
# FILTER (lang(?abstract) NOT IN("pt", "en"))

csv_results = conn.select(query, content_type='text/csv')
df = pd.read_csv(io.BytesIO(csv_results))
df


,thesis,abstract,lang
0,tag:stardog:api:com_a_condicao_de_servir_gratu...,A presente pesquisa parte da emancipação de es...,pt
1,tag:stardog:api:educando_libertos_escravizados...,Esta tese parte da trajetória de Cincinato Fra...,pt
2,tag:stardog:api:marambaia_historia_memoria_e_d...,A presente tese se desdobra em dois objetivos ...,pt
3,tag:stardog:api:mulheres_negras_politicas_iden...,O objetivo do presente estudo é conhecer as in...,pt
4,tag:stardog:api:os_sentidos_da_liberdade_traje...,Esta tese acompanhará trajetórias individuais ...,pt
5,tag:stardog:api:jose_do_patrocinio_marques_toc...,O propósito desta dissertação é refletir sobre...,pt
6,tag:stardog:api:a_escravidao_livre_na_corte_es...,"Durante a segunda metade do século XIX, na Cor...",pt
7,tag:stardog:api:ave_libertas_abolicionismos_e_...,O objetivo principal deste trabalho é compreen...,pt
8,tag:stardog:api:sou_da_macumba_e_no_feitico_na...,Pretende-se aqui discutir a vida e a obra do m...,pt
9,tag:stardog:api:a_gente_da_felisberta_conscien...,No presente trabalho tenho como objetivo anali...,pt


In [129]:
ex = df['abstract'][5]
ex

'O propósito desta dissertação é refletir sobre a histórica do afrodescendente em Goiás na segunda metade do século XIX, norteada pela experiência do jornalista e abolicionista goiano, José do Patrocínio Marques Tocantins; dentro do contexto socioeconômico, político e cultural da Província. Para iniciar, realizei uma análise das narrativas históricas já existentes e a visão dos viajantes sobre Goiás, no século XIX, as quais foram e continuam sendo referências para o estudo da História de Goiás; com atenção voltada para a influência dessas narrativas históricas na construção da representação do afrodescendente na memória coletiva da sociedade brasileira e estrangeira. Para (re) construir as experiências de José do Patrocínio, além de bibliografias, utilizei documentos e registros encontrados no Museu das Bandeiras (MUBAN), na Biblioteca Frei Simão e no arquivo diocesano Dom Tomás Baldoíno, ambos da Cidade de Goiás; e também os jornais: A Tribuna Livre e O Publicador Goyano, publicados n

In [130]:
#nlp_en = spacy.load("en_core_web_trf")
nlp_pt = spacy.load("pt_core_news_lg")
#doc = nlp_en(ex)
doc_pt = nlp_pt(ex)

persons = []
gpes = []
orgs = []
for ent in doc_pt.ents:
    if ent.label_ == "PER": #"PERSON":
        persons.append(ent)
    if ent.label_ == "LOC": #"GPE":
        gpes.append(ent)
    if ent.label_ == "ORG":
        orgs.append(ent)

    
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

#for ent in doc_pt.ents:
#    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Goiás 81 86 LOC
José do Patrocínio Marques Tocantins 184 220 PER
Província 280 289 LOC
Goiás 396 401 LOC
História de Goiás 480 497 MISC
José do Patrocínio 714 732 PER
Museu das Bandeiras 804 823 LOC
MUBAN 825 830 LOC
Biblioteca Frei Simão 836 857 LOC
Dom Tomás Baldoíno 881 899 PER
Cidade de Goiás 910 925 LOC
Tribuna Livre 950 963 MISC
O Publicador Goyano 966 985 MISC
Goiás 1270 1275 LOC
Anna Francisca Tocantins 1307 1331 PER
Província 1414 1423 LOC


In [131]:
persons, gpes, orgs

([José do Patrocínio Marques Tocantins,
  José do Patrocínio,
  Dom Tomás Baldoíno,
  Anna Francisca Tocantins],
 [Goiás,
  Província,
  Goiás,
  Museu das Bandeiras,
  MUBAN,
  Biblioteca Frei Simão,
  Cidade de Goiás,
  Goiás,
  Província],
 [])

In [132]:
for n in range(len(persons)-1):
    
    for m in range(n+1, len(persons)):
        print(n, m)
        print(persons[n], "---", persons[m], persons[n].similarity(persons[m]))


0 1
José do Patrocínio Marques Tocantins --- José do Patrocínio 0.9673900008201599
0 2
José do Patrocínio Marques Tocantins --- Dom Tomás Baldoíno 0.350134938955307
0 3
José do Patrocínio Marques Tocantins --- Anna Francisca Tocantins 0.4203408658504486
1 2
José do Patrocínio --- Dom Tomás Baldoíno 0.2963534891605377
1 3
José do Patrocínio --- Anna Francisca Tocantins 0.3132151663303375
2 3
Dom Tomás Baldoíno --- Anna Francisca Tocantins 0.3874610960483551


In [147]:
persons[0]

José do Patrocínio Marques Tocantins

In [16]:
#displacy.serve(doc_pt, style="ent")

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\spacy\displacy\__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



: 

Coletando dados da Wikidata

In [346]:
query = """ 
        
        SELECT ?person ?abstract
        WHERE{
        ?person rdfs:label 'José do Patrocínio'@pt.
        ?person a dbo:Person.
        ?person dbo:abstract ?abstract.
        FILTER (langMatches(lang(?abstract),"pt"))
        
        }
        """


In [347]:
import requests

url = 'http://dbpedia.org/sparql'

r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

In [348]:
data

{'head': {'link': [], 'vars': ['person', 'abstract']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'person': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Tomás_Balduino'},
    'abstract': {'type': 'literal',
     'xml:lang': 'pt',
     'value': 'Tomás Balduíno, O.P. (nome de nascimento Paulo Balduino de Sousa Décio, Posse, 31 de dezembro de 1922 — Goiânia, 2 de maio de 2014) foi um bispo e teólogo católico brasileiro, bispo-emérito de Goiás e assessor da Comissão Pastoral da Terra (CPT). Teve um papel de destaque dentro da Igreja Católica, nas questões referentes à reforma agrária e aos direitos dos povos indígenas. Participou da criação do Conselho Indigenista Missionário (Cimi), em 1972, da qual foi presidente entre 1980 e 1984, e da Comissão Pastoral da Terra, (CPT) em 1975, que presidiu de 1999 a 2005.'}}]}}